In [39]:
import pandas as pd
from scipy import stats
import numpy as np

In [40]:
# load the excel file
patient_data = pd.read_csv('../../data/KaggleV2-May-2016.csv')
patient_data.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [41]:
# Convert the No-show column from string to numeric (int)
patient_data['No-show'] = patient_data['No-show'].map({'No': 0, 'Yes': 1})

In [42]:
# Convert the dates to datetime object
patient_data['ScheduledDay'] = pd.to_datetime(patient_data['ScheduledDay']).dt.strftime('%Y-%m-%d')
patient_data['ScheduledDay'] = pd.to_datetime(patient_data['ScheduledDay'])
patient_data['AppointmentDay'] = pd.to_datetime(patient_data['AppointmentDay']).dt.strftime('%Y-%m-%d')
patient_data['AppointmentDay'] = pd.to_datetime(patient_data['AppointmentDay'])

In [43]:
# Determine the day of the week and create a categorical variable - Weekend or not, May be there are more no shows on weekends
patient_data['AppDay_weekend'] = patient_data['AppointmentDay'].apply(lambda x: 1 if x.dayofweek>=5 else 0)

In [44]:
# Determine the number of days between scheduled day and appointment day. may be more the gap, higher the chance of someone 
# not showing up
patient_data['DayDiff'] = (patient_data['AppointmentDay'] - patient_data['ScheduledDay']).dt.days

In [45]:
# Convert Gender to numeric
patient_data['Gender'] = patient_data['Gender'].map({'M': 0, 'F': 1})

In [46]:
# Check if same patient has missed previous appointment
patient_data['MissedApp'] = patient_data.groupby('PatientId')['No-show'].apply(lambda x: x.cumsum())

In [47]:
# Select only entries in the reasonable Age group. It's possible that there are entries which are just some people
# entering random values. So, the chances that other values are also incorrect is high
patient_data = patient_data[(patient_data['Age']>=0) & (patient_data['Age']<=100)]

In [48]:
patient_data.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,AppDay_weekend,DayDiff,MissedApp
0,2.987250e+13,5642903,1,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,0,0,0,0
1,5.589978e+14,5642503,0,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,0,0,0,0
2,4.262962e+12,5642549,1,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,0,0,0,0
3,8.679512e+11,5642828,1,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,0,0,0,0
4,8.841186e+12,5642494,1,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,0,0,0,0


In [49]:
# Drop unncessary columns
patient_data.drop(['PatientId', 'ScheduledDay', 'AppointmentDay', 'Neighbourhood'], axis=1, inplace=True)

In [50]:
#Set appointment Id as the index
patient_data.set_index('AppointmentID', inplace=True)

In [51]:
patient_data.head()

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,AppDay_weekend,DayDiff,MissedApp
AppointmentID,,,,,,,,,,,,
5642903,1,62,0,1,0,0,0,0,0,0,0,0
5642503,0,56,0,0,0,0,0,0,0,0,0,0
5642549,1,62,0,0,0,0,0,0,0,0,0,0
5642828,1,8,0,0,0,0,0,0,0,0,0,0
5642494,1,56,0,1,1,0,0,0,0,0,0,0


In [52]:
# Write the preprocessed data as a csv file
patient_data.to_csv('../../data/patient_data.csv', index=False)